# Zajęcia 1.

Zakres:
* zapoznanie sie z 2 srodowiskami pracy (klaster Hadoop oraz środowisko dostepne na GCP.
* operacje na rozproszonym i obiektowym systemie plików (HDFS i GCS)
* wykorzystanie platformy Apache Spark do interaktywnych kwerend na danych

Języki:
* bash, Python

Krótka lista przydatnych poleceń bash: https://www.reddit.com/r/linux/comments/9rns12/some_linux_commands_cheatsheet/

## Klaster Hadoop


URL Jupyter na klastrze Instytutu Informatyki: https://zsibio.ii.pw.edu.pl/jupyter/

### Architektura klastra
* edge node: cdh00 (węzeł dostępowy)
* HDFS:
  * name node: cdh01
  * data nodes: cdh02-cdh05
* YARN
  * resource manager: cdh01
  * node managers: cdh02-cdh05
* Dystrybucja Hadoop: Hortonworks
* konsola administracyjna: http://cdh01:8080/


### Jupyter Notebook

* Interaktywny notatnik dostepny poprzez przeglądarkę.
* Służy do wpisywania poleceń w wybranych jezykach programowania oraz opis w tzw jezyku markdown.
* Polecenia/opis wpisujemy do komórek. Komórki są wykonywane przez tzw. kernel (np Python w określonej wersji)
* Działa w tryb edycji komórek + tryb wykonywania poleceń. (przejście poprzez ESC i ENTER)
* Podstawowe skróty klawiaturowe: 
  * Esc/Enter
  * strzalki
  * ctrl-enter/shift-enter
  * a/b/d
  * y/m 
* Moze wykonywać polecenia powłoki (bash) !  %%bash
* Notatnik zapisywany jest w formacie ipynb (IPythonNoteBook)
* Kolejnosc uruchomienia komórek moze byc rozna.
* Mozliwosc zatrzymania kernela i uruchomienia od nowa. 

Polecamy film: https://www.youtube.com/watch?v=HW29067qVWk



### Security
Klaster jest zabezpieczony przed niepowolanym dostepem.
Zazwyczaj dostep do klastra to: zalogowanie poprzez ssh do wezla dostepowego a potem wykonywanie polecen. 
My logujemy sie do Jupytera, w tle pobierany jest ticket dostepowy do odpowiednich zasobów klastra. Korzystajac ze zmiennych srodowiskowych i plikow konfiguracyjnych Jupyter wie jak polaczyc sie z menedzerem zasobow oraz namenodem HDFS.

Dostep do Jupyter jest publiczny i po zajeciach mozna z niego korzystac.

## Kopiowanie danych do swojego katalogu domowego

W katalogu `/data/local/datascience/data/` znajduje sie plik który bedzie nam dzisiaj słuzyl do pracy. Należy go skopiować do swojego katalogu domowego. 
Dane pochodzą z https://gdac.broadinstitute.org/ i zawierają dane z badaniami nad mutacjami typu CNV w obrebie genu BRAC2.


In [ ]:
%%bash
cd                # przejdz do katalogu domowego
mkdir -p data     # stworz katalog data (i posrednie)
cd data           # przejdz do katalogu data 
cp /data/local/datascience/data/brca.txt .  # skopiuj plik z lokalizacji do bieżącej lokalizacji
ls -lah           # wylistuj zawartość bieżącego katalogu


In [ ]:
%%bash
head ~/data/brca.txt   # pokaż początek pliku
echo                   # pusta linia
tail ~/data/brca.txt   # pokaż koniec pliku 
echo
wc -l ~/data/brca.txt  # policz linie pliku


Do tej pory korzystalismy z lokalnego systemu plików do ktorego ma dostep maszyna na ktora sie zalogowalismy (cdh00). Teraz zaczniemy korzystac z rozproszonego systemu plikow.

## HDFS

* Dane są rozproszone na wielu serwerach i dzielone na bloki o ustalonym rozmiarze. 
* Bloki zostają rozdystrybuowane pomiędzy węzłami. Mogą być replikowane między węzłami.
* Name Node - odpowiedzialny za koordynację, zarządzanie metadanymi plików i aktywny monitoring replikacji plików.
* Data Node - węzeł składujący zawartość plików.
* Dostęp do danych, poprzez interfejs commandline, lub API programistyczne

Zakres na dzisiejszym lab:
* zapis i odczyt danych z HDFS
* manipulacje na plikach i podstawowe komendy

Dla podstawowych polecen systemowych na plikach istnieja odpowiedniki polecen dla systemu HDFS
* `ls` -> `hdfs dfs -ls`
* `cp` -> `hdfs dfs -cp`
* `mv` -> `hdfs dfs -mv`
* `rm` -> `hdfs dfs -rm`

Pełna lista poleceń znajduje się na www: https://hadoop.apache.org/docs/r2.4.1/hadoop-project-dist/hadoop-common/FileSystemShell.html

### Listowanie swojego HOME na HDFS
Każdy ma swoj katalog domowy na HDFS


In [ ]:
! hdfs dfs -ls    # wylistuj zawartość katalogu domowego (HDFS)

Swoj katalog mozna tez sprawdzic podajac sciezke bezwgledna od katalogu glownego

In [ ]:
! hdfs dfs -ls /user/${USER} # wylistuj zawartość katalogu domowego (HDFS)

Sprawdzmy katalog glowny na HDFS

In [ ]:
!hdfs dfs -ls / # wylistuj zawartość katalogu głównego (HDFS)

### **** ZADANIE 1 ****
Korzystajac z analogii do polecen znanych z lokalnego sytemu pliku oraz z dokumentacji polecen dla HDFS stworz w swoim katalogu domowym na HDFS katalog external a w nim katalog data. Sprawdz czy foldery istnieją.

### Dodanie pliku na HDFS

In [ ]:
%%bash
hdfs dfs -put ~/data/brca.txt /user/${USER}/external/data  # dodaj plik do wskazanej lokalizacji na HDFS
hdfs dfs -ls /user/${USER}/external/data                   # wylistuj zawartość katalogu na HDFS

Jaki jest rozmiar pliku? Mozna skorzystac z przelacznika -h w ls

In [ ]:
!hdfs dfs -ls -h /user/${USER}/external/data   # rozmiar pliku 

A calkowita wielkosc pliku? Polecenie `du`

In [ ]:
!hdfs dfs -du -h /user/${USER}/external/data/brca.txt  # całkowita przestrzeń zajmowana przez plik (replikacja)

Informacje o statusie pliku: polecenie `fsck`

In [ ]:
!hdfs fsck /user/${USER}/external/data/brca.txt

Plik mozna rowniez odczytac bezposrednio z HDFS:

In [ ]:
! hdfs dfs -head /user/${USER}/external/data/brca.txt # odczytaj początek pliku

### **** ZADANIE 2 ****
W swoim katalogu domowym stworz katalog external/temp. Skopiuj do niego plik brca.txt 

### Usuwanie i odzyskiwanie plikow

Do usuwanie sluzy polecenie -rm. Domyslnie usuwany plik jest przesuwany do kosza, skad mozna go odzyskac. 


In [ ]:
%%bash
hdfs dfs -rm /user/${USER}/external/temp/brca.txt # usun plik 
hdfs dfs -ls /user/${USER}/external/temp/          # wylistuj zawartość katalogu

Dostalismy lokalizacje pliku z kosza, stamtad mozemy go skopiowac.

Do samodzielnej weryfikacji:
* Przywrócenie pliku z kosza
* Zachowanie polecenia rm przy podaniu parametru `skipTrash`.
* modyfikacja uprawnień na katalogu poprzez polecenie `chmod`

### Pobranie pliku z powrotem na lokalny dysk

In [ ]:
!hdfs dfs -get /user/${USER}/external/data/brca.txt brca2.txt

## Apache Spark

* Apache Spark platforma ogólnego zastosowania, opensource, do przetwarzania duzych zbiorow danych.
* Posiada  API dla języków programowania: Scala, Python i R. 
* Przetwarzanie w Spark przetwarzanie jest wykonywane w większości  wprost w pamięci operacyjnej.
* Przeznaczenie: do uruchamiania  aplikacji i skryptów z wykorzystaniem uczenia maszynowego lub interaktywnych kwerend.
* Spark ten wspiera SQL (typ DataFrames), przetwarzanie strumieniowe oraz przetwarzanie grafów.
* Integracja z lokalną pamięci masową, rozproszonymi lub obiektowymi systemu plików.
* Spark można uruchamić na pojedynczej maszynie na środowisku klastrowym, lub w chmurze. 

Zakres na dzisiejsze laboratorium:
* stworzenie sesji Spark
* zaczytanie danych z pliku tekstowego
* kwerendy na danych

Na tych cwiczeniach bedziemy korzystac z pliku ktory zapisalismy na HDFS w pierwszej czesci cwicczenia.

In [ ]:
! hdfs dfs -ls /user/${USER}/external/data # Sprawdzmy czy plik znajduje się w katalogu

### Przygotowanie sesji Sparkowej
Losujemy numer portu. Domyslnie port na ktorym Spark wystawia swoj interfejs graficzny jest 4040. zeby nie zajmowac sobie wzajmnie numerow portu - wykonamy losowanie.
Nazwa aplikacji będzie udekorowana nazwą użytkownika który ją uruchamia.

In [ ]:
import os
user_name = os.environ.get('USER')
print(user_name)

In [ ]:
import random
ui_port = random.randint(4000,4999)
print(ui_port)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.master('yarn-client') \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '1g') \
.config('spark.ui.port',f'{ui_port}') \
.appName(f'app-{user_name}') \
.getOrCreate()

### Wczytanie danych


In [ ]:
path = f'/user/{user_name}/external/data/brca.txt'

In [ ]:
df = spark.read.load(path, format="csv", sep="\t", inferSchema="true", header="true")

### Charakterystyka danych

In [ ]:
type (df)                 # jaki jest typ danych

In [ ]:
df.explain()              # fizyczny plan wykonania

In [ ]:
df.explain(True)          # logiczny i fizyczny plan wykonania

In [ ]:
df.rdd.getNumPartitions() # liczba partycji (bloków danych)

In [ ]:
df.printSchema()          # schemat danych

In [ ]:
df.count()                # wymiary (liczba wierszy)

In [ ]:
len(df.columns)           # wymiary (liczba kolumn)

In [ ]:
df.describe().show()      # pokaż podsumowanie danych w tabeli

In [ ]:
df.describe("Chromosome").show() # pokaż podsumowanie danych w kolumnie 

### Odczyt danych

In [ ]:
df.show()                    # pokaż 20 wierszy danych 

In [ ]:
df.show(10, truncate=False) # pokaż 10 wierszy, nie skracaj danych

In [ ]:
df.select("Sample").show()  # pokaż tylko kolumne sample

In [ ]:
df.select("Chromosome", "Start", "End").show() # pokaż kolumny chromosome, start i end

In [ ]:
df_chrom = df.select(df.Chromosome).distinct() # stworz nowy DF zawierajacy tylko unikalne wartosci chromosomow

In [ ]:
df_chrom.count()                     # policz wiersze w nowym DF

In [ ]:
df.filter("Chromosome > 21").show() # pokaz dane spelniajace warunek

In [ ]:
df.filter("Chromosome > 21").explain()  # pokaz plan wykonania

In [ ]:
df.filter("Chromosome > 21 and Segment_Mean > 0").show() # pokaz dane spelniajace warunki

### Grupowanie i funkcje agregujące

In [ ]:
from pyspark.sql.functions import *

df.groupBy("Chromosome").count().show()   # dokonaj grupowania po chromosomie i policz rekordy w grupie

In [ ]:
df.groupBy("Chromosome").avg("Segment_Mean").show() # dokonaj grupowania po chromosomie i policz srednia wartosc segment_mean w grupie

In [ ]:
# pogrupuj dane wzgledem probki i chromosomu, policz rekordy w grupie
df.groupBy("Sample","Chromosome").count().orderBy(asc("Sample")).show()

### Kolumny wyliczane

In [ ]:
df.withColumn ("Length", col("End") - col("Start")).show() # dodaj kolumne dlugosc jako end - start

In [ ]:
df.withColumn ("Material", lit("DNA")).show()     # dodaj kolumne o stalej wartosci 'DNA'

In [ ]:
df.withColumn ("Chromosome2", concat(lit("chr"), col("Chromosome"))).show() # dodaj kolumne z konkatencja

In [ ]:
df.drop("Chromosome").show()  # usun kolumne Chromosome. 
# Czy DF została pozbawiona kolumny na trwale?

### **** Zadanie 3 *****

a) Ile jest unikalnych danych próbek w tym zbiorze danych?

b) Pokaż nazwę próbki, numer chromosomu, początek i koniec segmentu dla segmentów występujących na chromosomie 21,22,23 i mających startową pozycję większą niż 10000000. 

c) Ile występuje wierszy dla każdej z próbki? Pokaż wynik z pełną nazwą próbki.

# Przeniesienie pliku na GCS

In [ ]:
! gsutil cp ~/data/brca.txt gs://bucket-$USER  # upload obiektu do kubełka